# LangChain Multi-Query for RAG

## For implemantation see this notebook : <a href="https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/10-langchain-multi-query.ipynb" target="_blank">https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/10-langchain-multi-query.ipynb</a>

In [ ]:
# !pip install -qU \
#   pinecone-client==2.2.4 \
#   langchain==0.0.321 \
#   datasets==2.14.6 \
#   openai==0.28.1 \
#   tiktoken==0.5.1

In [ ]:
# from datasets import load_dataset

# data = load_dataset("jamescalam/ai-arxiv-chunked", split="train")
# data

In [2]:
# from langchain.docstore.document import Document

# docs = []

# for row in data:
#     doc = Document(
#         page_content=row["chunk"],
#         metadata={
#             "title": row["title"],
#             "source": row["source"],
#             "id": row["id"],
#             "chunk-id": row["chunk-id"],
#             "text": row["chunk"]
#         }
#     )
#     docs.append(doc)

## Embedding and Vector DB Setup

Initialize our embedding model:

In [6]:
import os
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings

# model_name = "text-embedding-ada-002"

# # get openai api key from platform.openai.com
# OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or getpass("OpenAI API Key: ")

# embed = OpenAIEmbeddings(
#     model=model_name, openai_api_key=OPENAI_API_KEY, disallowed_special=()
# )

Create our Pinecone index:

In [7]:
import pinecone
import time

# index_name = "langchain-multi-query-demo"

# # find API key in console at app.pinecone.io
# YOUR_API_KEY = os.getenv('PINECONE_API_KEY') or getpass("Pinecone API Key: ")
# # find ENV (cloud region) next to API key in console
# YOUR_ENV = os.getenv('PINECONE_ENVIRONMENT') or input("Pinecone Env: ")

# pinecone.init(
#     api_key=YOUR_API_KEY,
#     environment=YOUR_ENV
# )

# if index_name not in pinecone.list_indexes():
#     # we create a new index
#     pinecone.create_index(
#         name=index_name,
#         metric='cosine',
#         dimension=1536  # 1536 dim of text-embedding-ada-002
#     )
#     # wait for index to be initialized
#     while not pinecone.describe_index(index_name).status["ready"]:
#         time.sleep(1)

# # now connect to index
# index = pinecone.Index(index_name)


Populate our index:

In [8]:
# len(docs)

In [9]:
# if you want to speed things up to follow along
#docs = docs[:5000]

In [10]:
from tqdm.auto import tqdm
from uuid import uuid4

# batch_size = 100

# for i in tqdm(range(0, len(docs), batch_size)):
#     i_end = min(len(docs), i+batch_size)
#     docs_batch = docs[i:i_end]
#     # get IDs
#     ids = [f"{doc.metadata['id']}-{doc.metadata['chunk-id']}" for doc in docs_batch]
#     # get text and embed
#     texts = [d.page_content for d in docs_batch]
#     embeds = embed.embed_documents(texts=texts)
#     # get metadata
#     metadata = [d.metadata for d in docs_batch]
#     to_upsert = zip(ids, embeds, metadata)
#     index.upsert(vectors=to_upsert)
    
    

## Multi-Query with LangChain

Now we switch across to using our populated index as a vectorstore in Langchain.

In [11]:
# from langchain.vectorstores import Pinecone

# text_field = "text"

# vectorstore = Pinecone(index, embed.embed_query, text_field)

In [12]:
# from langchain.chat_models import ChatOpenAI

# llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

We initialize the `MultiQueryRetriever`:

In [13]:
from langchain.retrievers.multi_query import MultiQueryRetriever

# retriever = MultiQueryRetriever.from_llm(
#     retriever=vectorstore.as_retriever(), llm=llm
# )

In [14]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

To query with our multi-query retriever we call the `get_relevant_documents method`.

In [16]:
# question = "tell me about llama 2?"

# docs = retriever.get_relevant_documents(query=question)
# len(docs)

In [17]:
# docs

###  Adding the Generation in RAG

So far we've built a multi-query powered Retrieval Augmentation chain. Now, we need to add Generation.

In [18]:
import inspect

from getpass import getpass
from langchain import OpenAI
from langchain import PromptTemplate 

from langchain.callbacks import get_openai_callback

import tiktoken
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

In [19]:
import os
import openai

os.environ["OPENAI_API_TYPE"] = openai.api_type = "azure"
os.environ["OPENAI_API_VERSION"] = openai.api_version = "2022-12-01" #"2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = openai.api_base =  "https://idocopenaigpt.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = openai.api_key = "95776649ac7a4b048c834003fd315264"#os.getenv("AZUREOPENAI_API_KEY")

In [20]:
llm = AzureChatOpenAI(
    deployment_name="chatllm16k",
    openai_api_version="2023-03-15-preview",
    model_name="gpt-3.5-turbo",
    temperature=0,
)

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureChatOpenAI instead.
  warn_deprecated(
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:165: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://idocopenaigpt.openai.azure.com/ to https://idocopenaigpt.openai.azure.com/openai.
  warnings.warn(
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:172: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_na

In [21]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""You are a helpful assistant who answers user queries using the
    contexts provided. If the question cannot be answered using the information
    provided say "I don't know".

    Contexts:
    {contexts}

    Question: {query}""",
)

# Chain
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)

In [22]:
out = qa_chain(
    inputs={
        "query": question,
        "contexts": "\n---\n".join([d.page_content for d in docs])
    }
)
out["text"]

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'I\'m sorry, but without any information about "llama 2," I don\'t have enough context to provide an answer. Could you please provide more details or clarify your question?'

### Chaining Everything with a SequentialChain

We can pull together the logic above into a function or set of methods, whatever is prefered — however if we'd like to use LangChain's approach to this we must "chain" together multiple chains. The first retrieval component is (1) not a chain per se, and (2) requires processing of the output. To do that, and fit with LangChain's "chaining chains" approach, we setup the retrieval component within a `TransformChain`:

In [23]:
from langchain.chains import TransformChain

def retrieval_transform(inputs: dict) -> dict:
    docs = retriever.get_relevant_documents(query=inputs["question"])
    docs = [d.page_content for d in docs]
    docs_dict = {
        "query": inputs["question"],
        "contexts": "\n---\n".join(docs)
    }
    return docs_dict

retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts"],
    transform=retrieval_transform
)

Now we chain this with our generation step using the `SequentialChain`:

In [24]:

from langchain.chains import SequentialChain

rag_chain = SequentialChain(
    chains=[retrieval_chain, qa_chain],
    input_variables=["question"],  # we need to name differently to output "query"
    output_variables=["query", "contexts", "text"]
)

In [26]:
# Then we perform the full RAG pipeline:


# out = rag_chain({"question": question})
# out["text"]

## Custom Multiquery

We'll try this with two prompts, both encourage more variety in search queries.

**Prompt A**

    Your task is to generate 3 different search queries that aim to
    answer the user question from multiple perspectives.
    Each query MUST tackle the question from a different viewpoint,
    we want to get a variety of RELEVANT search results.
    Provide these alternative questions separated by newlines.
    Original question: {question}

__Prompt B__

    Your task is to generate 3 different search queries that aim to
    answer the user question from multiple perspectives. The user questions
    are focused on Large Language Models, Machine Learning, and related
    disciplines.
    Each query MUST tackle the question from a different viewpoint, we
    want to get a variety of RELEVANT search results.
    Provide these alternative questions separated by newlines.
    Original question: {question}

In [28]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser


# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

In [29]:
template = """
Your task is to generate 3 different search queries that aim to
answer the user question from multiple perspectives. The user questions
are focused on Large Language Models, Machine Learning, and related
disciplines.
Each query MUST tackle the question from a different viewpoint, we
want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.
Original question: {question}
"""

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template=template,
)


In [30]:
# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

In [32]:
# # Run
# retriever = MultiQueryRetriever(
#     retriever=vectorstore.as_retriever(), llm_chain=llm_chain, parser_key="lines"
# )  # "lines" is the key (attribute name) of the parsed output

# # Results
# docs = retriever.get_relevant_documents(
#     query=question
# )

In [33]:
len(docs)
docs

[]

In [ ]:
# retrieval_chain = TransformChain(
#     input_variables=["question"],
#     output_variables=["query", "contexts"],
#     transform=retrieval_transform
# )

# rag_chain = SequentialChain(
#     chains=[retrieval_chain, qa_chain],
#     input_variables=["question"],  # we need to name differently to output "query"
#     output_variables=["query", "contexts", "text"]
# )

And asking again:

In [34]:
# out = rag_chain({"question": question})
# out["text"]

In [35]:
# pinecone.delete_index(index_name)